# NOP NOP Trajectories Bug Testing

Includes functionality to:
1. Read encounter information straight from the bin files
2. Simulate encounter files
3. Visualize those simulations (if not small_simulation_output())

**Make sure you are in a Julia 1.1 Env!
And that you have aircraftshapes.sty in repo**

In [ ]:
include("src/Xr.jl");

### 1. Reading straight from the encounter files

In [ ]:
open("data_files/nop_nop_U*U*_flip_test_1k.bin", "r") do f
    dt = read(f, Float64)
    num_steps = read(f, Int64)
    # For each encounter
    for i = 1:2
        # Make an encounter out of info
        curr_phys_state = read_phys_state(f)
        ẍ = read_vector(f, num_steps)
        ÿ = read_vector(f, num_steps)
        z̈ = read_vector(f, num_steps)

        #@show curr_phys_state
        #@show ẍ
        #@show ÿ
        #@show z̈
        #@show length(ẍ)
    end
end


### 2. Simulating the encounter files

Make sure this notebook is within the UAMEncounterGen repo. Make sure that your encounter files are in the "data_files" folder within UAMEncounterGenerator. 

When running a simulation, to see alerts or nmacs, must do sim.sim_out = small_simulation_output()

When running a simulation, to be able to visualize it, do not do a small simulation

sim 1 : trajectory 1 equipped as ac1

sim 2 : trajectory 1 equipped as ac2

sim 3 : trajectory 2 equipped as ac2

sim 4 : trajectory 2 equipped as ac1

In [162]:
sim1 = simulation()

# comment this back in when wanting to see the alert counts
# sim1.sim_out = small_simulation_output()

sim1.enc_file = "data_files/nop_nop_U*U*_flip_test_1l.bin"
sim1.acs[1] = uam_speed(q_file = "data_files/test_speed_120.bin")
sim1.acs[1].perform_scaling = true

# turn responsive to false when wanting to check if trajectories are being properly generated
# sim1.acs[1].responsive = false

xr_sim!(sim1);

In [ ]:
sim2 = simulation()
sim2.sim_out = small_simulation_output()
sim2.enc_file = "data_files/nop_nop_U*U*_flip_test_2l.bin"
sim2.acs[2] = uam_speed(q_file = "data_files/test_speed_120.bin")
sim2.acs[2].perform_scaling = true
xr_sim!(sim2);

In [ ]:
sim3 = simulation()
sim3.sim_out = small_simulation_output()
sim3.enc_file = "data_files/nop_nop_U*U*_flip_test_1l.bin"
sim3.acs[2] = uam_speed(q_file = "data_files/test_speed_120.bin")
sim3.acs[2].perform_scaling = true
xr_sim!(sim3);

In [ ]:
sim4 = simulation()
sim4.sim_out = small_simulation_output()
sim4.enc_file = "data_files/nop_nop_U*U*_flip_test_2l.bin"
sim4.acs[1] = uam_speed(q_file = "data_files/test_speed_120.bin")
sim4.acs[1].perform_scaling = true
xr_sim!(sim4);

In [ ]:
println(sim1.sim_out.alerts)
println(sim2.sim_out.alerts)
println()
println(sim3.sim_out.alerts)
println(sim4.sim_out.alerts)

### 3. Visualizing the simulated encounter sets

Make sure the simulation you want to visualize does not have sim.sim_out = small_simulation_output()

ac1 = UAM icon

ac2 = quad icon

In [ ]:
include("visualization/UAMEncounterViewer.jl");

In [ ]:
encounter_viewer(sim1.sim_out, int_type=:quad, alert_type=:speed)